<h1><b>Machine Learning Based Clustering and Segmentation for Navigation<b></h1>

<h3><b>Introduction</b></h3>
    <p>
    An ML based navigation algorithm that is based on several factors pertaining to neighbourhoods. That will give you the most efficient route to the desired destination, based on factors such as crime rate and population density.
    </p>
<h3><b>Project Contribution</b></h3>
    <p>
    The project contribution is to find correlations between topics surrounding the crime rate, population information and income sources. The purpose of this Jupyter notebook is to focus on the following correlations:
        <ul>
            <li>Correlation between police stations and assault rates</li>
            <li>Correlation between schools and income</li>
        </ul>
    </p>
<h3><b>Prerequisites</b></h3>
<ul>
    <li>Foursquare API</li>
</ul>
<h3><b>Datasets Used</b></h3>

<h3><b>Import Statements</b></h3>

In [2219]:
from dotenv import load_dotenv
from dotenv import dotenv_values
import folium
import requests
import pandas as pd 
from pandas import json_normalize
from bs4 import BeautifulSoup as bs
import os
from sklearn.cluster import KMeans

import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors


<h3><b>Foursquare API Initialization / Check</b></h3>
<h4><b>Category Codes:</b></h4>
<ul>
    <li>10000 - Arts and Entertainment</li>
    <li>11000 - Business and Professional Services</li>
    <li>12000 - Community and Government</li>
    <li>13000 - Dining and Drinking</li>
    <li>14000 - Event</li>
    <li>15000 - Health and Medicine</li>
    <li>16000 - Landmarks and Outdoors</li>
    <li>17000 - Retail</li>
    <li>18000 - Sports and Recreation</li>
    <li>19000 - Travel and Transportation</li>
</ul>

In [2220]:
config = dotenv_values(".env")
url = "https://api.foursquare.com/v3/places/nearby"

headers = {"Accept": "application/json",
            "Authorization": config["API_KEY"]}

response = requests.request("GET", url, headers=headers)

def create_request(coords= None, location = None, categories = None, limit = "10"):
    """
        Important:
            - Coords and location cannot be entered together
            - Location and radius cannot be entered together

        The coords will be a list with latitude and longitude.\n 
        Location will be a city and province such as  "Oshawa, ON".\n
        The category is a string from the above codes, with a default of None.\n
        The limit parameter is a maximum of 50, with a default of 10 requests.\n

        Examples:
            - create_request(coords=[-72.848752,43.895962], limit="1")
            - create_request(coords=[-72.848752,43.895962], categories="10000", limit="2")\n
            - create_request(location=["Oshawa","ON"], limit="2")
            - create_request(location=["Oshawa","ON"], categories="10000", limit="20")
    """

    if(coords and categories == None):
        url = "https://api.foursquare.com/v3/places/search?ll=" + str(coords[0]) + "%2C" + str(coords[1]) + "&radius=100000"  + "&limit=" + limit
    elif(coords and categories):
        url = "https://api.foursquare.com/v3/places/search?ll=" + str(coords[0]) + "%2C" + str(coords[1]) +"&categories=" + categories + "&radius=100000" + "&limit=" + limit
    elif(location and categories == None):
        url = "https://api.foursquare.com/v3/places/search?" + "near=" + str(location[0]) + "%2C" + str(location[1]) + "&limit=" + limit
    elif(location and categories):
        url = "https://api.foursquare.com/v3/places/search?" + "categories=" + categories + "&near=" + str(location[0]) + "%2C" + str(location[1]) + "&limit=" + limit
    else:
        return False
    
    response = requests.request("GET", url, headers=headers)
    
    if(response.status_code == 200):
        return response.json()
    else:
        return False

<h3><b>Creating Law Enforcement DataFrame</b></h3>

In [2221]:
latitude = 43.6532
longitude = -79.3832
police_stations = create_request(location=["Toronto","ON"], categories="12070", limit="50")

# Generates data from the Foursquare API
police_venues = json_normalize(police_stations['results'], max_level=3)
pd.DataFrame(police_venues)
police_venues.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,related_places.children
0,4c00016237850f473c8e973f,"[{'id': 12072, 'name': 'Police Station', 'icon...",[],1441,/v3/places/4c00016237850f473c8e973f,53 Division Toronto Police Service,America/Toronto,43.706104,-79.400647,43.706104,-79.400647,75 Eglinton Ave W,CA,at Duplex Ave.,"75 Eglinton Ave W (at Duplex Ave.), Toronto ON...",Toronto,[Yonge and Eglinton],M4R 2G9,ON,NaN
1,4cc5d24f80624688058a3e2f,"[{'id': 12072, 'name': 'Police Station', 'icon...",[],1693,/v3/places/4cc5d24f80624688058a3e2f,Toronto Police Service - 13 Division,America/Toronto,43.698433,-79.436581,43.698433,-79.436581,1435 Eglinton Ave W,CA,Allen Expressway,"1435 Eglinton Ave W (Allen Expressway), Toront...",Toronto,[York],M6C 3Z4,ON,NaN
2,4e700aa1ae602b2e721fc1c7,"[{'id': 12071, 'name': 'Fire Station', 'icon':...",[],2106,/v3/places/4e700aa1ae602b2e721fc1c7,Toronto Fire Station 341,America/Toronto,43.694398,-79.441081,43.694398,-79.441081,555 Oakwood Ave,CA,,"555 Oakwood Ave, Toronto ON M6E 2X4",Toronto,NaN,M6E 2X4,ON,NaN
3,4d83b00b7e8ef04d20d8fbbd,"[{'id': 12071, 'name': 'Fire Station', 'icon':...",[],3084,/v3/places/4d83b00b7e8ef04d20d8fbbd,Toronto Fire Station 131,America/Toronto,43.726175,-79.402729,43.726175,-79.402729,3135 Yonge St,CA,Wanless Ave,"3135 Yonge St (Wanless Ave), Toronto ON M4N 2K8",Toronto,NaN,M4N 2K8,ON,NaN
4,4dcd5207922e8ac4247be29d,"[{'id': 12072, 'name': 'Police Station', 'icon...",[],4088,/v3/places/4dcd5207922e8ac4247be29d,University of Toronto Campus Community Police,America/Toronto,43.664817,-79.400841,43.664817,-79.400841,21 Sussex Ave,CA,btwn Huron & St. George,"21 Sussex Ave (btwn Huron & St. George), Toron...",Toronto,[Susex Ulster],M5S 1J6,ON,NaN


<h3><b>Creating Educational DataFrame</b></h3>

In [2222]:
latitude = 43.6532
longitude = -79.3832
educational = create_request(location=["Toronto","ON"], categories="12057", limit="50")

# Generates data from the Foursquare API
education_venues = json_normalize(educational['results'], max_level=3)
pd.DataFrame(education_venues)
education_venues.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,...,location.cross_street,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,location.address_extended,related_places.children,related_places.parent.fsq_id,related_places.parent.name
0,4bb10197f964a5202f723ce3,"[{'id': 12059, 'name': 'High School', 'icon': ...",[],535,/v3/places/4bb10197f964a5202f723ce3,Forest Hill CI,America/Toronto,43.702022,-79.421244,43.702022,...,,"730 Eglinton Ave W, Toronto ON M5N 1B9",Toronto,[York],M5N 1B9,ON,NaN,NaN,NaN,NaN
1,4b61f1b7f964a5209b2c2ae3,"[{'id': 12058, 'name': 'Elementary School', 'i...",[],596,/v3/places/4b61f1b7f964a5209b2c2ae3,Forest Hill Jr & Sr PS,America/Toronto,43.695270,-79.413705,43.695270,...,,"78 Dunloe Rd, Toronto ON M5P 2T6",Toronto,[York],M5P 2T6,ON,NaN,NaN,NaN,NaN
2,4addb992f964a5207e6521e3,"[{'id': 12058, 'name': 'Elementary School', 'i...",[],1253,/v3/places/4addb992f964a5207e6521e3,Bishop Strachan School,America/Toronto,43.689576,-79.409379,43.689576,...,Lonsdale east of Spadina,"298 Lonsdale Rd (Lonsdale east of Spadina), To...",Toronto,[Forest Hill South],M4V 1X2,ON,NaN,NaN,NaN,NaN
3,4ad4c064f964a52069f820e3,"[{'id': 12057, 'name': 'Primary and Secondary ...",[],1360,/v3/places/4ad4c064f964a52069f820e3,Upper Canada College,America/Toronto,43.690935,-79.403045,43.690935,...,NaN,"200 Lonsdale Rd, Toronto ON M4V 1W6",Toronto,[Midtown],M4V 1W6,ON,# 234,NaN,NaN,NaN
4,4dbc5de0fa8cee72734b5c6d,"[{'id': 12058, 'name': 'Elementary School', 'i...",[],1391,/v3/places/4dbc5de0fa8cee72734b5c6d,Allenby Junior Public School,America/Toronto,43.709718,-79.412740,43.709718,...,NaN,"391 Saint Clements Ave, Toronto ON M5N 1M2",Toronto,[Yonge and Eglinton],M5N 1M2,ON,NaN,"[{'fsq_id': '5733ad6d498ed068d25eef81', 'name'...",NaN,NaN


<h3><b>Gather the Postal Codes</b></h3>

In [2223]:
path = os.getcwd()
path = os.path.join(path,"datasets/neighborhood-data.csv")
postcodes = pd.read_csv(path)
postcodes.drop(postcodes.columns[[0]], axis=1, inplace=True)
postcodes.head()



,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


<h3><b>Gather the Dataframe for Homicide Rates and Crime Rates</b></h3>

In [2224]:
path2 = os.getcwd()
path2 = os.path.join(path2,"datasets/neighbourhood-crime-rates.csv")
crimedata = pd.read_csv(path2)
crimedata.drop(crimedata.columns[[0]], axis=1, inplace=True)

path3 = os.getcwd()
path3 = os.path.join(path3,"datasets/combined-dataset.csv")
combined_data = pd.read_csv(path3)
combined_data.drop(combined_data.columns[[0]], axis=1, inplace=True)
combined_data.head()

,Postcode,Borough,Neighbourhood,Hood_ID,Population,Assault_AVG,Assault_Rate_2019,AutoTheft_AVG,AutoTheft_Rate_2019,Homicide_AVG,Homicide_Rate_2019,Latitude,Longitude
0,M3A,North York,Parkwoods,45,34805,159.7,454.0,31.5,91.9,0.3,2.9,43.751,-79.323
1,M4A,North York,Victoria Village,43,17510,119.3,753.9,16.5,102.8,0.7,5.7,43.735,-79.312
2,M6A,North York,Lawrence Heights,32,22372,104.0,518.5,28.5,102.8,0.2,0.0,43.722,-79.451
3,M1B,Scarborough,Rouge,131,46496,173.3,391.4,50.5,187.1,0.8,0.0,43.804,-79.165
4,M1B,Scarborough,Malvern,132,43794,278.2,760.4,47.2,162.1,1.7,2.3,43.809,-79.221


<h3><b></b></h3>


In [2225]:
k = 6
toronto_clustering = combined_data.drop(columns=["Postcode", "Population","Borough", "Neighbourhood", "Assault_AVG", "Assault_Rate_2019", "AutoTheft_Rate_2019", "Homicide_AVG" , "Homicide_Rate_2019", "Latitude", "Longitude"])
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
combined_data.insert(0, "Cluster Labels", kmeans.labels_)
combined_data

,Cluster Labels,Postcode,Borough,Neighbourhood,Hood_ID,Population,Assault_AVG,Assault_Rate_2019,AutoTheft_AVG,AutoTheft_Rate_2019,Homicide_AVG,Homicide_Rate_2019,Latitude,Longitude
0,5,M3A,North York,Parkwoods,45,34805,159.7,454.0,31.5,91.9,0.3,2.9,43.751,-79.323
1,5,M4A,North York,Victoria Village,43,17510,119.3,753.9,16.5,102.8,0.7,5.7,43.735,-79.312
2,5,M6A,North York,Lawrence Heights,32,22372,104.0,518.5,28.5,102.8,0.2,0.0,43.722,-79.451
3,4,M1B,Scarborough,Rouge,131,46496,173.3,391.4,50.5,187.1,0.8,0.0,43.804,-79.165
4,4,M1B,Scarborough,Malvern,132,43794,278.2,760.4,47.2,162.1,1.7,2.3,43.809,-79.221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2,M4X,Downtown Toronto,Cabbagetown,71,11669,102.3,1079.8,10.7,188.5,0.3,0.0,43.665,-79.368
79,2,M4X,Downtown Toronto,St. James Town,71,11669,102.3,1079.8,10.7,188.5,0.3,0.0,43.670,-79.373
80,0,M8Y,Etobicoke,Kingsway Park South East,15,9271,25.8,302.0,16.5,302.0,0.0,0.0,43.619,-79.500
81,0,M8Y,Etobicoke,Mimico NE,17,33964,299.2,959.8,37.3,176.7,0.7,2.9,43.614,-79.495


<h3><b>Correlation #1: Law Enforcement Buildings in Neighbourhoods and Auto Theft Rates</b></h3>

In [2226]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for average, lat, lon, neighbourhood, cluster in zip(combined_data["AutoTheft_AVG"], combined_data['Latitude'], combined_data['Longitude'], combined_data['Neighbourhood'], combined_data['Cluster Labels']):
    label = folium.Popup('Neighbourhood ' + str(neighbourhood) + " Auto Theft " + str(average), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

for name, latitude, longitude in zip(police_venues['name'], police_venues['geocodes.main.latitude'], education_venues['geocodes.main.longitude']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [latitude,longitude],
    radius=5,
    popup=label,
    color='black',
    fill=True,
    fill_color='black',
    fill_opacity=0.7,
    parse_html=False).add_to(map_clusters)

       
map_clusters

<h3><b>Gather the Dataframe for Income Rates and Education Rates</b></h3>

In [2227]:
path4 = os.getcwd()
path4 = os.path.join(path4,"datasets/population-dataset-combined.csv")
population_data = pd.read_csv(path4)
population_data.drop(population_data.columns[[0]], axis=1, inplace=True)
population_data.head()

,Postcode,Borough,Neighbourhood,Neighbourhood Number,"Total - Highest certificate, diploma or degree for the population aged 15 years and over in private households - 25% sample data","No certificate, diploma or degree",Secondary (high) school diploma or equivalency certificate,Trades certificate or diploma other than Certificate of Apprenticeship or Certificate of Qualification,Certificate of Apprenticeship or Certificate of Qualification,"College, CEGEP or other non-university certificate or diploma",...,"$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over","$200,000 and over",Latitude,Longitude
0,M3A,North York,Parkwoods,45,28890,4140,7660,700,605,5295,...,620,"1,200","1,025",880,790,650,"3,795",890,43.751,-79.323
1,M4A,North York,Victoria Village,43,14550,2930,3780,475,260,2930,...,425,640,570,490,385,290,"1,445",240,43.735,-79.312
2,M6A,North York,Lawrence Manor,32,17080,2675,4340,505,330,2635,...,335,735,565,500,435,315,"2,155",755,43.726,-79.436
3,M9A,Etobicoke,Islington Avenue,14,37405,4440,8415,880,790,6590,...,755,"1,530","1,475","1,365","1,170","1,065","6,445",1665,43.643,-79.529
4,M1B,Scarborough,Rouge,131,38125,6580,11740,1020,635,7740,...,455,970,950,930,940,845,"6,060",1075,43.805,-79.166


In [2228]:
k = 6
toronto_clustering = population_data[["Neighbourhood Number", "$5,000 to $9,999"]]
toronto_clustering
# toronto_clustering = population_data.drop(columns = ["Postcode"])
# toronto_clustering = toronto_clustering.drop(toronto_clustering.columns[[2,3]])
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
# kmeans.labels_
population_data.insert(0, "Cluster Labels", kmeans.labels_)
population_data.head()


,Cluster Labels,Postcode,Borough,Neighbourhood,Neighbourhood Number,"Total - Highest certificate, diploma or degree for the population aged 15 years and over in private households - 25% sample data","No certificate, diploma or degree",Secondary (high) school diploma or equivalency certificate,Trades certificate or diploma other than Certificate of Apprenticeship or Certificate of Qualification,Certificate of Apprenticeship or Certificate of Qualification,...,"$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over","$200,000 and over",Latitude,Longitude
0,3,M3A,North York,Parkwoods,45,28890,4140,7660,700,605,...,620,"1,200","1,025",880,790,650,"3,795",890,43.751,-79.323
1,3,M4A,North York,Victoria Village,43,14550,2930,3780,475,260,...,425,640,570,490,385,290,"1,445",240,43.735,-79.312
2,3,M6A,North York,Lawrence Manor,32,17080,2675,4340,505,330,...,335,735,565,500,435,315,"2,155",755,43.726,-79.436
3,5,M9A,Etobicoke,Islington Avenue,14,37405,4440,8415,880,790,...,755,"1,530","1,475","1,365","1,170","1,065","6,445",1665,43.643,-79.529
4,0,M1B,Scarborough,Rouge,131,38125,6580,11740,1020,635,...,455,970,950,930,940,845,"6,060",1075,43.805,-79.166


<h3><b>Correlation #2: Educational Buildings and Income </b></h3>


In [2229]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for average, lat, lon, neighbourhood, cluster in zip(population_data["$200,000 and over"], population_data['Latitude'], population_data['Longitude'], population_data['Neighbourhood'], population_data['Cluster Labels']):
    label = folium.Popup('Neighbourhood: ' + str(neighbourhood) + " Income: " + str(average), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

for name, latitude, longitude in zip(education_venues['name'], education_venues['geocodes.main.latitude'], education_venues['geocodes.main.longitude']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [latitude,longitude],
    radius=5,
    popup=label,
    color='black',
    fill=True,
    fill_color='black',
    fill_opacity=0.7,
    parse_html=False).add_to(map_clusters)

map_clusters

<h3><b>Correlation #3: </b></h3>
